# Decision trees 

## Recursion

A recursive function is a function that calls itself until some base case is reached. The base case is a condition we check with every call to the function to make sure it still makes sense to call itself. Without the base case the recursion would continue infinitely.

Recursion is often explained by referring to Russian nesting dolls. Each time you open a doll, another doll is inside. This continues until you reach the smallest doll (the base case). Without knowing how many dolls there are we know how to solve the task of opening all the dolls, as we simply keep calling the open *'function'* until we reach the last doll.

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d2/Russian-Matroshka_no_bg.jpg" width="30%">

An example of a problem we can solve using a recursive function is calculating the factorial. The base case is that if ```n == 1``` we no longer need to calculate the factorial, as here we know the answer, and otherwise we calculate the answer by calculating the factorial for ```n-1```, until we reach 1. In the cell below the ```factorial``` function is given, with print statements to show what's happening.

In [1]:
def factorial(n):
    if n == 1:
        print("This I know! (the base case)")
        return 1
    else:
        print("I don't know the factorial for", n, "let's try", n-1)
        return n * factorial(n-1)
    
factorial(5)

I don't know the factorial for 5 let's try 4
I don't know the factorial for 4 let's try 3
I don't know the factorial for 3 let's try 2
I don't know the factorial for 2 let's try 1
This I know! (the base case)


120

Let's practice a bit with recursion.

### Exercise 1

Write a recursive function in the cell below which takes a list of numbers and returns the sum of that list. 

**Hint:** Remember that you can use the a colon to select a part of a list. For example ```a[2:]``` returns all but the first two elements from the list ```a```.

In [1]:
def rec_sum(a):
    if len(a) == 1:
        return a[0]
    else:
        return a[0] + rec_sum(a[1:])
   
    
rec_sum([1,2,3,4,5,6])

21

Clearly the function in exercise 1 is not the most useful recursive function, and it can be easily solved with a loop. But it might help you get started thinking about how it works. Let's see how it can be more useful.

### Exercise 2

In the cell below you are given a list, which contains a nested list, which contains another nested list, etc. You do not know how many nested lists there are, all you know that the last list contains a number. Write a recursive function which prints this number by searching through the list.

Another variant is to also keep track of how many levels you had to descend in order to reach the final answer.

**Hint:** You can check if something is a list using the ```isinstance``` function: ```isinstance([1,2,3], list) == True```

In [2]:
nested = [[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[[13]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]

def search(a, depth=0):
    if isinstance(a, list):
        return search(a[0], depth+1)
    else:
        return a, depth
    
search(nested)

(13, 37)

## Tree structures

Recursive functions are very useful when dealing with tree structures,  which are recursive structures themselves. We do not know how deep the tree is. All we can see is if the node we are currently looking at has any children, and if it does we can try to visit those, and repeat this.

A tree splits up into branches (child nodes). Decision trees are usually full binary trees which means that every node has either 0 or 2 children. If it has 0 then it is a leaf node. The figure below is annotated with some of the terminology used when talking about trees.


<img src="https://www.tutorialspoint.com/data_structures_algorithms/images/binary_tree.jpg" alt="Tree structure" width="50%">

The cell below defines a class ```Node```, which we can use to construct a Decision tree. The nodes stores references to its children (left and right) and to the question (feature value) to ask about at this node, as well as the class we'll predict at this node.


In [6]:
class Node:
    """A node in a binary decision tree"""
    
    def __init__(self, left=None, right=None, feature=None, value=None, predict=None):
        """Initialize the node with attributes."""
        self.left = left
        self.right = right
        self.feature = feature # column in which features is stored
        self.value = value # value to check against
        self.predict = predict # class to predict at this node
        
    def isLeaf(self):
        """Helper function to check if the current node is a leaf"""
        return self.left is None and self.right is None
       
    def __str__(self, depth=1):
        """ You can ignore this function, 
        but basically it helps print the node in a human-readable manner """
        if self.isLeaf():
            return "Predict: \"{:s}\"".format(self.predict)
        else:
            s = "if features[{:d}] != \"{:s}\" then:\n {:s} \n{:s}else:\n {:s}"
            return s.format(self.feature, 
                            self.value, 
                            "\t" * depth+self.left.__str__(depth+1),
                            "\t" * (depth-1),
                            "\t" * depth+self.right.__str__(depth+1))

Let's illustrate the use of this class with an example of a made-up tree below.

In [7]:
# We want to first ask about value Round in column at index 2.
root = Node(feature=2, value="Round",         
            
            # If false, in the left branch, which is a leaf node, 
            # we'll we'll predict Banana  
            left=Node(predict="Banana"),
            
            # If true, in the right branch we'll ask about the color Red
            right=Node(feature=1, value="Red", 
                       
                       # Based on the answer to question about color Red, 
                       # we'll predict either Lime
                       left=Node(predict="Lime"),
                       
                       # or Apple
                       right=Node(predict="Apple")))

# Thanks to the __str__() function we can print the tree 
# and get the rules formatted in a humanly readable format.
print(root)

if features[2] != "Round" then:
 	Predict: "Banana" 
else:
 	if features[1] != "Red" then:
 		Predict: "Lime" 
	else:
 		Predict: "Apple"


In [8]:
# Additionally we can use isleaf() to check if a node is a leaf node or not.
print("Is root a leaf node?", root.isLeaf())

print("Is the right child of root a leaf node?", root.right.isLeaf())
print()

# If we want to find out which column the root looks at we can:
print("The root looks at column", root.feature, 
      "and checks if its value is equal or not to", root.value)

Is root a leaf node? False
Is the right child of root a leaf node? False

The root looks at column 2 and checks if its value is equal or not to Round


### Dataset

In the example above we made up the decisions, but normally you would want to generate these based on the data. For this we'll use the weather dataset in the next cell, the objective of this dataset is to figure out if the weather conditions are such that it is nice enough to go and play outside. 

It has the following features, all of which are categorical.
- outlook {sunny, overcast, rainy}
- temperature {hot, mild, cool}
- humidity {high, normal}
- windy {TRUE, FALSE}

And the target is:
- Can we play outside today? {yes, no}

The features are stored in X_train, each row in X_train is a different day/moment. y_train contains the label for each row.

In [5]:
X_train = [['sunny', 'hot', 'high', 'FALSE'],
 ['sunny', 'hot', 'high', 'TRUE'],
 ['overcast', 'hot', 'high', 'FALSE'],
 ['rainy', 'mild', 'high', 'FALSE'],
 ['rainy', 'cool', 'normal', 'FALSE'],
 ['rainy', 'cool', 'normal', 'TRUE'],
 ['overcast', 'cool', 'normal', 'TRUE'],
 ['sunny', 'mild', 'high', 'FALSE'],
 ['sunny', 'cool', 'normal', 'FALSE'],
 ['rainy', 'mild', 'normal', 'FALSE'],
 ['sunny', 'mild', 'normal', 'TRUE'],
 ['overcast', 'mild', 'high', 'TRUE'],
 ['overcast', 'hot', 'normal', 'FALSE'],
 ['rainy', 'mild', 'high', 'TRUE']]

y_train = ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']

Let's do some quick analysis of the distribution of the features and the label, and write a function which will be useful later on.

### Exercise 3

Write a function in the next cell called ```majority``` which takes a list of categorical values and returns which occurs most often.

**Hint:** {'A': 5, 'B': 6}.items() returns an iterator of pairs of tuples, which you can sort using ```sorted()```.

In [18]:
def majority(a):
    counts = {}
    for i in a:
        if i in counts:
            counts[i] += 1
        else:
            counts[i] = 0
    #print(counts)
    return sorted(counts.items(), key=lambda x: x[1], reverse=True)[0][0]

In [19]:
if majority(y_train) == 'yes' and majority(y_train[:3]) == 'no':
    print("Majority is correct!")
else:
    print("Your majority function contains a mistake")

Majority is correct!


## Generating and evaluating potential decisions

Now that we have a dataset we can figure out which questions to ask. To do this we first need to generate the set of potential questions. Because we are dealing with features which are categorical all our questions are going to be whether the feature's value is equal to a particular (of the form ```if temperature == 'hot'```). 

### Exercise 4

Write a function ```question_set``` which ```X_train``` as input and returns the unique values in each column. One way to do this is to generate a list where the nth row contains the set of unique values in the $n$th column.

You shouldn't need a recursive function to solve this.

**Hint:** the **set** of potential questions.

In [13]:
def question_set(X):
    samples = len(X)
    features = len(X[0])
    qset = [set() for col in range(features)]

    for f in range(features):
        for s in range(samples):
            qset[f].add(X[s][f])
            
    return qset

question_set(X_train)

[{'overcast', 'rainy', 'sunny'},
 {'cool', 'hot', 'mild'},
 {'high', 'normal'},
 {'FALSE', 'TRUE'}]

Before we continue to determining whether a question is a good one to ask or not, let's figure out how to actually apply one to a dataset. Or in others words if we have a question how do we split the dataset according to the answer.

### Exercise 5

Write the function ```split``` in the cell below that takes the node defined below, ```X_train```, and ```y_train```, and returns four lists. The first containing the rows from X_train which have outlook == overcast, the second containing the labels for those rows. The third and fourth lists should contain the same but then for the rows which have a different outlook.

**Hints** 
- The easiest way to do this is probably by creating the four lists at the start of your function, appending to them when appropriate and then returning them at the end.

In [14]:
def split(feature, value, X, y):
    yes, no = [], []
    yes_labels, no_labels = [], []
    
    for i in range(len(X)):
        if X[i][feature] == value:
            yes.append(X[i])
            yes_labels.append(y[i])
        else:
            no.append(X[i])
            no_labels.append(y[i])
            
    return no, no_labels, yes, yes_labels
    
split(0, 'overcast', X_train, y_train)

([['sunny', 'hot', 'high', 'FALSE'],
  ['sunny', 'hot', 'high', 'TRUE'],
  ['rainy', 'mild', 'high', 'FALSE'],
  ['rainy', 'cool', 'normal', 'FALSE'],
  ['rainy', 'cool', 'normal', 'TRUE'],
  ['sunny', 'mild', 'high', 'FALSE'],
  ['sunny', 'cool', 'normal', 'FALSE'],
  ['rainy', 'mild', 'normal', 'FALSE'],
  ['sunny', 'mild', 'normal', 'TRUE'],
  ['rainy', 'mild', 'high', 'TRUE']],
 ['no', 'no', 'yes', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no'],
 [['overcast', 'hot', 'high', 'FALSE'],
  ['overcast', 'cool', 'normal', 'TRUE'],
  ['overcast', 'mild', 'high', 'TRUE'],
  ['overcast', 'hot', 'normal', 'FALSE']],
 ['yes', 'yes', 'yes', 'yes'])

Once we know how to split the dataset based on questions, we need decide which questions to ask first. In the lecture we discussed how the best decisions reduces the uncertainty the most. So let's write some functions to help us measure the uncertainty.

## Entropy

Entropy is a measure of uncertainty, Entropy is calculated as follows:

$H(P) = - \sum\limits_{i=1}^N P_i \log_2(P_i)$

where $P$ is a list of class probabilities (i.e., the proportion the class is present in the set). Given that for decision trees you'll be dealing with lists of labels you'll need to convert these to probabilities for each individual label. 

### Exercise 6

Write the ```entropy``` function in the cell below. Add tests to verify that the entropy of the list ```[0,1]``` is `1.0`, of the list ```[-1,1,2,3]``` is 2.0, and that the entropy of the first 10 examples of y_train is higher than that of the whole y_train.

In [15]:
from math import log2

def entropy(labels):
    counts = {}
    for label in labels:
        counts[label] = counts.get(label, 0) + 1/len(labels)
    #print(counts)
    return -sum([p*log2(p) for p in counts.values()])

if not (entropy([0,1]) == 1.0 and
        entropy([-1,1,2,3]) == 2.0 and
        entropy(y_train[:10]) > entropy(y_train)):
    print("Your entropy function contains a mistake!")

## Information gain

However, just measuring the entropy at a certain point in the tree isn't enough. We need to know how much the entropy goes down if we make a certain decision. For decision trees a commonly used measure is Information Gain. The information gain is calculated by subtracting the entropy of the weighted sum of the child nodes from the entropy of the parent node. Typically weighting is done by the relative size of the children.

$IG(P) = H(P) - \sum\limits_{i=1}^N w_i~H(C_i)$

Here $N$ is the number of children, $C_i$ is the ith child, and $w_i$ is the weight given to the ith child. Given that we'll be dealing with decision trees which only have a left and a right child we can also write this out to be:

$IG(P) = H(P) - w_{left}~H(C_{left}) - w_{right}~H(C_{right})$

Usually the weight (i.e., $w_{left}$) is equal to the proportion the child node has of the parent node. For example, if the parent contains $20$ instances, and after the decision the left child would have 15 and the right 5, then $w_{left} = \frac{15}{20}$ and $w_{right} = \frac{5}{20}$.

### Exercise 7

Implement the information gain function in the cell below. To verify if you have done it correctly you can split ```X_train``` and ```y_train``` using the function you wrote for exercise 5 using node A and B. If correct, the information gain using node A should be higher than when using node B. The function should take the list of labels in the left branch and a list of labels in the right branch.


In [16]:
def IG(left, right):
    parent = left + right
    w_left = len(left) / len(parent)
    w_right = len(right) / len(parent)
    return entropy(parent) - w_left * entropy(left) - w_right * entropy(right)   

L, yL, R, yR = split(0, 'overcast', X_train, y_train)
print(IG(yL,yR))

L, yL, R, yR = split(0, 'sunny', X_train, y_train)
print(IG(yL,yR))

0.22600024438491684
0.10224356360985082


We have all the building blocks we need to start fitting a tree to a dataset. Let's give it a go!

## Advanced

### Exercise 8

Implement the ```fit(X,y)``` function below, where X is a matrix of features and y is a list of labels. It should return a tree (i.e., a instance of the Node() class).

**Hints:** 
- Start by thinking of what the right base case is, and implement this.
- Remember that this should be a recursive function.

In [20]:
def fit(X, y):
    if entropy(y) == 0:
        return Node(predict=majority(y))
    else:
        qs = question_set(X)
        scores = []
        for feature, row in enumerate(qs):
            for value in row:
                _, yleft, _, yright = split(feature, value, X, y)
                scores.append((IG(yleft, yright), feature, value))
                
        bestIG, feature, value = sorted(scores, key = lambda x: x[0])[-1]
        
        Xleft, yleft, Xright, yright = split( feature, value, X, y)
        
        left = fit(Xleft, yleft)
        right = fit(Xright, yright)
        return Node(feature=feature, value=value, left=left, right=right)
    
decision_tree = fit(X_train, y_train)
print(decision_tree)

if features[0] != "overcast" then:
 	if features[2] != "normal" then:
 		if features[0] != "rainy" then:
 			Predict: "no" 
		else:
 			if features[3] != "TRUE" then:
 				Predict: "yes" 
			else:
 				Predict: "no" 
	else:
 		if features[3] != "TRUE" then:
 			Predict: "yes" 
		else:
 			if features[1] != "mild" then:
 				Predict: "no" 
			else:
 				Predict: "yes" 
else:
 	Predict: "yes"


### Exercise 9

Once we have fitted a decision tree we would like to verify how well it works, and use it to predict the label for new samples. Implement the ```predict``` function in the cell below, where ```tree``` is a fitted tree, and ```x``` is one feature vector (a list). It should return a single label, either 'yes' or 'no'.

**Hints:**
- What is the base case?
- Remember that going left or right depends on the decision at each node.

In [21]:
def predict(tree, x):
    if tree.isLeaf():
        return tree.predict
    elif x[tree.feature] != tree.value:
        return predict(tree.left, x)
    else:
        return predict(tree.right, x)

print('\t\tData\t\t\tTruth\tPrediction')
for row, label in zip(X_train, y_train):
    print(row, '\t', label, '\t', predict(decision_tree, row))

		Data			Truth	Prediction
['sunny', 'hot', 'high', 'FALSE'] 	 no 	 no
['sunny', 'hot', 'high', 'TRUE'] 	 no 	 no
['overcast', 'hot', 'high', 'FALSE'] 	 yes 	 yes
['rainy', 'mild', 'high', 'FALSE'] 	 yes 	 yes
['rainy', 'cool', 'normal', 'FALSE'] 	 yes 	 yes
['rainy', 'cool', 'normal', 'TRUE'] 	 no 	 no
['overcast', 'cool', 'normal', 'TRUE'] 	 yes 	 yes
['sunny', 'mild', 'high', 'FALSE'] 	 no 	 no
['sunny', 'cool', 'normal', 'FALSE'] 	 yes 	 yes
['rainy', 'mild', 'normal', 'FALSE'] 	 yes 	 yes
['sunny', 'mild', 'normal', 'TRUE'] 	 yes 	 yes
['overcast', 'mild', 'high', 'TRUE'] 	 yes 	 yes
['overcast', 'hot', 'normal', 'FALSE'] 	 yes 	 yes
['rainy', 'mild', 'high', 'TRUE'] 	 no 	 no
